In [1]:
import numpy as np
import pandas as pd

In [2]:
N = 11

### Матрицы

In [3]:
def determinant(matrix, mul):
    width = len(matrix)
    if width == 1:
        return mul * matrix[0][0]
    else:
        sign = -1
        s = 0
        for i in range(width):
            m = []
            for j in range(1, width):
                buff = []
                for k in range(width):
                    if k != i:
                        buff.append(matrix[j][k])
                m.append(buff)
            sign *= -1
            s += mul * determinant(m, sign * matrix[0][i])
    return s

In [4]:
class Array:
    
    def __init__(self, a):
        self.array = np.array(a, dtype = 'float')
        self.shape = self.shape_f()

    def shape_f(self):
        return self.array.shape
    
    def T(self):
        res = np.zeros((self.shape[1], self.shape[0]))
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                res[j][i] = self.array[i][j]
        return Array(res)
        

    def plus(self, other):
        res = np.array(self.array)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                res[i][j] += other.array[i][j]
        return Array(res)

    def pointwise_multiply(self, other):
        res = np.array(self.array)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                res[i][j] *= other.array[i][j]
        return Array(res)
    

    def matrix_multiply(self, other):
        res = np.zeros((self.shape[0], other.shape[1]))
        for i in range(self.shape[0]):
            for j in range(other.shape[1]):
                for k in range(len(self.array[i])):
                    res[i][j] += self.array[i][k] * other.array[k][j]
        return Array(res)
    def det(self):
        return determinant(self.array, 1)
        
    
    # Operators
    def __add__(self, other):
        return self.plus(other)
    
    def __sub__(self, other):
        return self.plus(Array(-(other.array)))
    
    def __mul__(self, other):
        return self.pointwise_multiply(other)
    
    def __matmul__(self, other):
        return self.matrix_multiply(other)
    
    def __getitem__(self, ij):
        (i, j) = ij
        return self.array[i][j]

### Нормы

In [5]:
def sqrt(x, eps = 10**(-9), delta=1):
    p = (1+x)/2 
    p1 = (p+x/p)/2
    while abs(p-p1) > eps/delta:
        p = p1
        p1 = (p+x/p)/2
    return p1

def v_norm_inf(v):
    return max(v.array)
def v_norm_1(v):
    return sum(abs(v.array))
def v_norm_2(v):
    return np.sqrt((v.T() @ v).array[0][0])

def m_norm_inf(A):
    return max(sum(abs(A.array)))
def m_norm_1(A):
    return m_norm_inf(A.T())
def m_norm_2(A):
    M = A.T() @ A
    return sqrt(max(np.linalg.eigh(M.array)[0]))
def m_norm_f(A):
    return sum(sum((A*A).array))
    

m_norms = [m_norm_1, m_norm_inf]
v_norms = [v_norm_1, v_norm_2, v_norm_inf]
m_n_norm = 0
v_n_norm = 0

# Метод простой итерации

In [6]:
def simple_iteration(A, b, eps):
    if not A.det() :
        print("Матрица А особая")
        return 0
    v_n_norm = 0
    b = b.T()
    n = A.shape[0]
    mu = 1/m_norms[m_n_norm](A)
    Mu = Array(np.ones(A.shape) * mu)
    B = Array(np.eye(n)) - Mu*A
    flag = 0
    if m_norms[0](B)>1 and m_norms[1](B)>1:
        b = A.T() @ b
        A = A.T() @ A
        mu = 1/m_norms[m_n_norm](A) 
        Mu = Array(np.ones(A.shape) * mu)
        B = Array(np.eye(n)) - Mu*A
        if m_norms[0](B)>1 and m_norms[1](B)>1:
            flag = 1
            if m_norms[0](B)>1: v_n_norm = 1
            else: v_n_norm = 0
        
    Mu = Array(np.ones(b.shape) * mu)
    c = Mu * b
    x_old = c
    x = B @ x_old + c
    
    B_norm = m_norms[m_n_norm](B)/(1-m_norms[m_n_norm](B))
    iterations = 1
    if flag:
        while v_norms[v_n_norm](A @ x - b) > eps:
            x_old = x
            x = B @ x_old + c
            iterations +=1
    else:
        while B_norm * v_norms[v_n_norm](x-x_old) > eps:
            x_old = x
            x = B @ x_old + c
            iterations +=1
    return x, iterations

# Метод Зейделя

In [7]:
def seidel(A, b, eps):
    if not A.det() :
        print("Матрица А особая")
        return 0
    n = A.shape[0]
    b = b.T()
    
    M = [abs(A[i, i]) > (sum(abs(A.array[i]))-abs(A[i, i])) for i in range(n)]
    if sum(M) != n:
        b = A.T() @ b
        A = A.T() @ A
    C = Array(A.array)
    C = Array([C.array[i]*(-1/A[i, i]) for i in range(n)]) + Array(np.eye(n))
    d = Array([[b[i, 0]/A[i, i] for i in range(n)]]).T()
    x_old = Array(d.array)
    x = Array(x_old.array)
    for i in range(n):
        x_old = Array(x.array)
        x.array[i] = (Array([C.array[i]]) @ x_old)[0, 0] + d[i, 0]
    iterations = 1
    while v_norms[v_n_norm](A @ x - b) > eps:
        for i in range(n):
            x_old = Array(x.array)
            x.array[i] = (Array([C.array[i]]) @ x_old)[0, 0] + d[i, 0]
        iterations += 1
    return x, iterations

# Метод Гаусса

In [8]:
def gauss(A, b):
    if not A.det() :
        print("Матрица А особая")
        return 0
    A = Array(A.array)
    n = A.shape[0]
    b = b.T()
    for i in range(n):
        T = A.T()
        m = np.argmax(abs(T.array[i][i:]))
        A.array[i], A.array[i+m] = np.array(A.array[i+m]), np.array(A.array[i])
        b.array[i], b.array[i+m] = np.array(b.array[i+m]), np.array(b.array[i])
        a = A[i, i]
        A.array[i] = A.array[i]*(1/a)
        b.array[i] = b.array[i]*(1/a)
        for j in range(i+1, n):
            b.array[j] = np.array(b.array[j] - A[j, i] * b.array[i])
            A.array[j] = np.array(A.array[j] - A[j, i] * A.array[i])
    x = Array(np.zeros(b.shape))
    x.array[n-1] = b.array[n-1]
    for i in range(n-1):
        tmp = Array(-x.array)
        x.array[n-2-i] = (Array([A.array[n-2-i]]) @ tmp)[0,0] + b.array[n-2-i]
    return x

# Метод Хаусхолдера

In [9]:
def householder(A, b):
    if not A.det() :
        print("Матрица А особая")
        return 0
    A = Array(A.array)
    n = A.shape[0]
    b = b.T()
    
    Q = Array(np.eye(n))
    Ri = Array(A.array)
    R = [Ri]
    
    for i in range(n-1):
        Q_old = Array(np.eye(n))
        R_old = Array(Ri.array[i:, i:])
        zi = Array(np.zeros((n-i, 1)))
        zi.array[0][0] = 1
        yi = Array([R_old.array[:, 0]]).T()
        alpha = v_norm_2(yi)
        alp = Array([np.ones(zi.shape) * alpha]).T()
        ro = v_norm_2(yi-alp*zi)
        wi = yi - alp*zi
        wi = Array([np.ones(wi.shape)* (1/ro)]).T() * wi
        Qi = Array(np.eye(n-i)) - wi @ wi.T() - wi @ wi.T()
        Q_old.array[i:, i:] = Qi.array
        Ri.array[i:, i:] = (Qi @ R_old).array
        Q = Q_old @ Q
        R.append(R[i])
    y = Q @ b
    x = gauss(R[-1], y.T())
    
    return x

# Выводы

In [10]:
# Example
A_t = Array([[3, 1, 1], [1, 5, 1], [1, 1, 7]])
b_t = Array([[5, 7, 9]])

In [11]:
# Example 2
A_t2 = Array([[0, 1, 1], [1, 5, 1], [1, 1, 7]])
b_t2 = Array([[2, 20, 12]])

In [12]:
# Test 0
A0 = Array([[0, 2, 3], [1, 2, 4], [4, 5, 6]])
b0 = Array([[13, 17, 32]])

In [13]:
# Test 1
A1 = Array([[N+2, 1, 1], [1, N+4, 1], [1, 1, N+6]])
b1 = Array([[N+4, N+6, N+8]])

In [14]:
# Test 2
A2 = Array([[-(N+2), 1, 1], [1, -(N+4), 1], [1, 1, -(N+6)]])
b2 = Array([[-(N+4), -(N+6), -(N+8)]])

In [15]:
# Test 3
A3 = Array([[-(N+2), N+3, N+4], [N+5, -(N+4), N+1], [N+4, N+5, -(N+6)]])
b3 = Array([[N+4, N+6, N+8]])

In [16]:
# Test 4
A4 = Array([[N+2, N+1, N+1], [N+1, N+4, N+1], [N+1, N+1, N+1]])
b4 = Array([[N+4, N+6, N+8]])

In [17]:
A = [A0, A1, A2, A3, A4]
b = [b0, b1, b2, b3, b4]

In [18]:
nt = 5
tests = list(range(nt))

epsilons = [10**(-i) for i in range(2, 5)]

## Метод простой итерации

In [19]:
x_my = []
k = []
x_true = []
errors = []
epss = []
testss = []
for i in tests:
    for epsilon in epsilons:
        x_my.append(simple_iteration(A[i], b[i], epsilon)[0])
        k.append(simple_iteration(A[i], b[i], epsilon)[1])
        x_true.append(Array([np.linalg.solve(A[i].array, b[i].T().array)]))
        epss.append(epsilon)
        testss.append(i)
for i in range(len(x_true)):
    errors.append(v_norm_1(x_my[i] - x_true[i].T()))

In [20]:
d = {
    'тест': testss,
    'Х настоящий': [x.T().array for x in x_true],
    'эпсилон': epss,
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors,
    'итерации': k
}

In [21]:
pd.DataFrame(data=d)

,тест,Х настоящий,эпсилон,Х посчитанный,погрешность,итерации
0,0,"[[1.0], [2.0], [3.0]]",0.0100,"[[1.0020681202414385, 2.0011733100152553, 2.99...",[0.005121843533523407],189
1,0,"[[1.0], [2.0], [3.0]]",0.0010,"[[0.9999349272455449, 2.0006951179613006, 2.99...",[0.0012414507108751138],278
2,0,"[[1.0], [2.0], [3.0]]",0.0001,"[[0.999885909563963, 2.000252625559777, 2.9998...",[0.0004972667727629609],670
3,1,"[[1.0], [1.0], [1.0]]",0.0100,"[[0.9978361123686899, 1.0004103228505805, 1.00...",[0.0030451124934824936],4
4,1,"[[1.0], [1.0], [1.0]]",0.0010,"[[0.9997528616351078, 1.0000678844596516, 1.00...",[0.0003591253487428281],6
5,1,"[[1.0], [1.0], [1.0]]",0.0001,"[[0.9999714378513238, 1.0000089353266945, 1.00...",[4.208241000025481e-05],8
6,2,"[[1.3525091799265605], [1.3084455324357405], [...",0.0100,"[[1.3501938547695436, 1.3073541602481082, 1.27...",[0.004119837771357604],11
7,2,"[[1.3525091799265605], [1.3084455324357405], [...",0.0010,"[[1.352256975303324, 1.3083268041460836, 1.274...",[0.0004485616370151213],15
8,2,"[[1.3525091799265605], [1.3084455324357405], [...",0.0001,"[[1.352481714150235, 1.3084326045043015, 1.274...",[4.884703565055659e-05],19
9,3,"[[1.2997485067588808], [1.1188305564287961], [...",0.0100,"[[1.2974593315001028, 1.11669563769657, 1.0797...",[0.006867644680438767],31


## Метод Зейделя

In [22]:
x_my = []
k = []
x_true = []
errors = []
epss = []
testss = []
for i in tests:
    for epsilon in epsilons:
        x_my.append(seidel(A[i], b[i], epsilon)[0])
        k.append(seidel(A[i], b[i], epsilon)[1])
        x_true.append(Array([np.linalg.solve(A[i].array, b[i].T().array)]))
        epss.append(epsilon)
        testss.append(i)
for i in range(len(x_true)):
    errors.append(v_norm_1(x_my[i] - x_true[i].T()))

In [23]:
d = {
    'тест': testss,
    'Х настоящий': [x.T().array for x in x_true],
    'эпсилон': epss,
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors,
    'итерации': k
}

In [24]:
pd.DataFrame(data=d)

,тест,Х настоящий,эпсилон,Х посчитанный,погрешность,итерации
0,0,"[[1.0], [2.0], [3.0]]",0.0100,"[[1.008295446337934, 1.9823516599217292, 3.008...",[0.034865990776561606],222
1,0,"[[1.0], [2.0], [3.0]]",0.0010,"[[1.0008244107253361, 1.998246088245013, 3.000...",[0.0034650211181923574],347
2,0,"[[1.0], [2.0], [3.0]]",0.0001,"[[1.0000834582121838, 1.9998224448871111, 3.00...",[0.0003507771779576174],471
3,1,"[[1.0], [1.0], [1.0]]",0.0100,"[[1.0003872884575573, 0.9998727602113282, 0.99...",[0.0005298252267517123],2
4,1,"[[1.0], [1.0], [1.0]]",0.0010,"[[1.000010964366861, 1.0000002888409107, 0.999...",[1.1915161170161781e-05],3
5,1,"[[1.0], [1.0], [1.0]]",0.0001,"[[1.0000000287009605, 1.0000000422168291, 0.99...",[7.508942423406495e-08],4
6,2,"[[1.3525091799265605], [1.3084455324357405], [...",0.0100,"[[1.3524861667201384, 1.3084388017655775, 1.27...",[3.1493516383873654e-05],3
7,2,"[[1.3525091799265605], [1.3084455324357405], [...",0.0010,"[[1.3524861667201384, 1.3084388017655775, 1.27...",[3.1493516383873654e-05],3
8,2,"[[1.3525091799265605], [1.3084455324357405], [...",0.0001,"[[1.352508527595025, 1.3084453723043181, 1.274...",[8.602548962066692e-07],4
9,3,"[[1.2997485067588808], [1.1188305564287961], [...",0.0100,"[[1.2997375600847207, 1.1188221467688362, 1.08...",[2.6741409566266583e-05],16


## Метод Гаусса

In [25]:
x_my = []
x_true = []
errors = []
testss = []
for i in tests:
    x_my.append(gauss(A[i], b[i]))
    x_true.append(Array([np.linalg.solve(A[i].array, b[i].T().array)]))
    testss.append(i)
for i in range(len(x_true)):
    errors.append(v_norm_1(x_my[i] - x_true[i].T()))

In [26]:
d = {
    'тест': testss,
    'Х настоящий': [x.T().array for x in x_true],
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors
}

In [27]:
pd.DataFrame(data=d)

,тест,Х настоящий,Х посчитанный,погрешность
0,0,"[[1.0], [2.0], [3.0]]","[[1.0, 2.0, 3.0]]",[0.0]
1,1,"[[1.0], [1.0], [1.0]]","[[1.0, 1.0, 1.0]]",[0.0]
2,2,"[[1.3525091799265605], [1.3084455324357405], [...","[[1.3525091799265607, 1.3084455324357405, 1.27...",[2.220446049250313e-16]
3,3,"[[1.2997485067588808], [1.1188305564287961], [...","[[1.2997485067588808, 1.1188305564287961, 1.08...",[0.0]
4,4,"[[-3.999999999999993], [-0.666666666666665], [...","[[-3.9999999999999982, -0.6666666666666665, 6....",[1.3988810110276972e-14]


## Метод Хаусхолдера

In [28]:
x_my = []
x_true = []
errors = []
testss = []
for i in tests:
    x_my.append(householder(A[i], b[i]))
    x_true.append(Array([np.linalg.solve(A[i].array, b[i].T().array)]))
    testss.append(i)
for i in range(len(x_true)):
    errors.append(v_norm_1(x_my[i] - x_true[i].T()))

In [29]:
d = {
    'тест': testss,
    'Х настоящий': [x.T().array for x in x_true],
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors
}

In [30]:
pd.DataFrame(data=d)

,тест,Х настоящий,Х посчитанный,погрешность
0,0,"[[1.0], [2.0], [3.0]]","[[0.9999999999999982, 2.0000000000000036, 2.99...",[7.549516567451064e-15]
1,1,"[[1.0], [1.0], [1.0]]","[[1.0000000000000002, 0.9999999999999993, 0.99...",[1.7763568394002505e-15]
2,2,"[[1.3525091799265605], [1.3084455324357405], [...","[[1.3525091799265605, 1.3084455324357405, 1.27...",[2.220446049250313e-16]
3,3,"[[1.2997485067588808], [1.1188305564287961], [...","[[1.2997485067588808, 1.118830556428796, 1.082...",[4.440892098500626e-16]
4,4,"[[-3.999999999999993], [-0.666666666666665], [...","[[-3.9999999999998987, -0.6666666666666674, 6....",[1.8984813721090177e-13]


### Test 5

In [31]:
def bad_matr(n, eps):
    E = Array(np.eye(n))
    A = Array(np.zeros((n,n)))
    B = Array(np.ones((n,n)) * eps * N)
    for i in range(n):
        for j in range(n):
            if i < j:
                A.array[i][j] = -1
                B.array[i][j] = B.array[i][j] * (-1)
    return E + A + B

In [32]:
A = []
B = []
es = []
ns = []
for n in range(4, 7):
    for e in [10**(-3), 10**(-6)]:
        A.append(bad_matr(n, e))
        B.append(np.ones(n)*(-1))
        es.append(e)
        ns.append(n)
for bi in B:
    bi[-1] = 1
b = [Array([Bi]) for Bi in B]

## Метод простой итерации

In [33]:
x_my = []
k = []
x_true = []
errors = []
epss = []
size = []
ess = []
for i in range(len(A)):
    for epsilon in epsilons:
        x_my.append(simple_iteration(A[i], b[i], epsilon)[0])
        k.append(simple_iteration(A[i], b[i], epsilon)[1])
        x_true.append(Array([np.linalg.solve(A[i].array, b[i].T().array)]))
        epss.append(epsilon)
        size.append(ns[i])
        ess.append(es[i])

for i in range(len(x_true)):
    errors.append(v_norm_1(x_my[i] - x_true[i].T()))

In [34]:
d = {
    'размерность': size,
    'e': ess,
    'Х настоящий': [x.T().array for x in x_true],
    'эпсилон': epss,
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors,
    'итерации': k
}

In [35]:
pd.DataFrame(data=d)

,размерность,e,Х настоящий,эпсилон,Х посчитанный,погрешность,итерации
0,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...",0.0100,"[[-0.12501192691041718, -0.0630318007443337, -...",[0.24480977247981708],10
1,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...",0.0010,"[[-0.019900418109179696, -0.010019281756036913...",[0.039042654238595986],272
2,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...",0.0001,"[[-0.0019829854772022426, -0.00099837551679649...",[0.003890421594248916],601
3,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...",0.0100,"[[-0.1265818856458789, -0.06435791750508069, -...",[0.24809070695360397],9
4,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...",0.0010,"[[-0.02548395769078335, -0.0129579862655965, -...",[0.050117677073936114],296
5,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...",0.0001,"[[-0.002561120555702512, -0.001302268877856927...",[0.005036792735083457],708
6,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...",0.0100,"[[-0.07127887091042939, -0.035788419743476386,...",[0.14161802408846416],15
7,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...",0.0010,"[[-0.05896677352498137, -0.029305211823672534,...",[0.11723440103343195],135
8,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...",0.0001,"[[-0.005900679990362201, -0.002932510406865195...",[0.011731397921355255],1588
9,5,0.000001,"[[1.110198600255951e-16], [1.110198600255951e-...",0.0100,"[[-0.06801442714086449, -0.03447746589410053, ...",[0.1353092137063912],15


## Метод Зейделя

In [36]:
x_my = []
k = []
x_true = []
errors = []
epss = []
size = []
ess = []
for i in range(len(A)):
    for epsilon in epsilons:
        x_my.append(seidel(A[i], b[i], epsilon)[0])
        k.append(seidel(A[i], b[i], epsilon)[1])
        x_true.append(Array([np.linalg.solve(A[i].array, b[i].T().array)]))
        epss.append(epsilon)
        size.append(ns[i])
        ess.append(es[i])

for i in range(len(x_true)):
    errors.append(v_norm_1(x_my[i] - x_true[i].T()))

In [37]:
d = {
    'размерность': size,
    'e': ess,
    'Х настоящий': [x.T().array for x in x_true],
    'эпсилон': epss,
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors,
    'итерации': k
}

In [38]:
pd.DataFrame(data=d)

,размерность,e,Х настоящий,эпсилон,Х посчитанный,погрешность,итерации
0,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...",0.0100,"[[0.13003195315237992, 0.06408404423651369, 0....",[0.25271354835426246],17
1,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...",0.0010,"[[0.012995687761728547, 0.006404704453147632, ...",[0.025256764110295555],54
2,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...",0.0001,"[[0.0012988184542801484, 0.000640100661886936,...",[0.0025242181809320367],91
3,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...",0.0100,"[[0.16875712072981974, 0.08437736955490541, 0....",[0.32941851965400604],16
4,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...",0.0010,"[[0.01673827897850466, 0.008369021378028467, 0...",[0.032673578802535734],63
5,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...",0.0001,"[[0.0017438601458967806, 0.0008719177676536618...",[0.0034040627456823893],109
6,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...",0.0100,"[[0.31347363315542, 0.15467503790063045, 0.078...",[0.6210995138270718],43
7,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...",0.0010,"[[0.03146608095611181, 0.015526081780721367, 0...",[0.06234517202343154],151
8,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...",0.0001,"[[0.003158524819994124, 0.001558488161587909, ...",[0.006258128348347944],259
9,5,0.000001,"[[1.110198600255951e-16], [1.110198600255951e-...",0.0100,"[[0.5455774811522904, 0.272785249171282, 0.139...",[1.0842063362982068],28


## Метод Гаусса

In [39]:
x_my = []
x_true = []
errors = []
size = []
ess = []
for i in range(len(A)):
    for epsilon in epsilons:
        x_my.append(gauss(A[i], b[i]))
        x_true.append(Array([np.linalg.solve(A[i].array, b[i].T().array)]))
        size.append(ns[i])
        ess.append(es[i])

for i in range(len(x_true)):
    errors.append(v_norm_1(x_my[i] - x_true[i].T()))

In [40]:
d = {
    'размерность': size,
    'e': ess,
    'Х настоящий': [x.T().array for x in x_true],
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors,
}

In [41]:
pd.DataFrame(data=d)

,размерность,e,Х настоящий,Х посчитанный,погрешность
0,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...","[[-4.440892098500626e-16, -2.220446049250313e-...",[8.299952300281778e-16]
1,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...","[[-4.440892098500626e-16, -2.220446049250313e-...",[8.299952300281778e-16]
2,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...","[[-4.440892098500626e-16, -2.220446049250313e-...",[8.299952300281778e-16]
3,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...","[[4.440892098500626e-16, 2.220446049250313e-16...",[1.3322432060406704e-15]
4,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...","[[4.440892098500626e-16, 2.220446049250313e-16...",[1.3322432060406704e-15]
5,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...","[[4.440892098500626e-16, 2.220446049250313e-16...",[1.3322432060406704e-15]
6,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...","[[-4.440892098500626e-16, -2.220446049250313e-...",[7.559324064669662e-16]
7,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...","[[-4.440892098500626e-16, -2.220446049250313e-...",[7.559324064669662e-16]
8,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...","[[-4.440892098500626e-16, -2.220446049250313e-...",[7.559324064669662e-16]
9,5,0.000001,"[[1.110198600255951e-16], [1.110198600255951e-...","[[4.440892098500626e-16, 2.220446049250313e-16...",[5.551163971864194e-16]


## Метод Хаусхолдера

In [42]:
x_my = []
x_true = []
errors = []
size = []
ess = []
for i in range(len(A)):
    for epsilon in epsilons:
        x_my.append(householder(A[i], b[i]))
        x_true.append(Array([np.linalg.solve(A[i].array, b[i].T().array)]))
        size.append(ns[i])
        ess.append(es[i])

for i in range(len(x_true)):
    errors.append(v_norm_1(x_my[i] - x_true[i].T()))

In [43]:
d = {
    'размерность': size,
    'e': ess,
    'Х настоящий': [x.T().array for x in x_true],
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors,
}

In [44]:
pd.DataFrame(data=d)

,размерность,e,Х настоящий,Х посчитанный,погрешность
0,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...","[[1.3322676295501878e-15, 6.661338147750939e-1...",[4.1606643039036475e-15]
1,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...","[[1.3322676295501878e-15, 6.661338147750939e-1...",[4.1606643039036475e-15]
2,4,0.001000,"[[-8.578788670727691e-16], [-4.254672515606004...","[[1.3322676295501878e-15, 6.661338147750939e-1...",[4.1606643039036475e-15]
3,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...","[[-3.3306690738754696e-16, -2.220446049250313e...",[4.441038639342806e-16]
4,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...","[[-3.3306690738754696e-16, -2.220446049250313e...",[4.441038639342806e-16]
5,4,0.000001,"[[-2.2203239317027257e-16], [-1.11014975420406...","[[-3.3306690738754696e-16, -2.220446049250313e...",[4.441038639342806e-16]
6,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...","[[-4.884981308350689e-15, -2.6645352591003757e...",[8.458918697921833e-15]
7,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...","[[-4.884981308350689e-15, -2.6645352591003757e...",[8.458918697921833e-15]
8,5,0.001000,"[[-8.214514341878545e-16], [-4.068542987038759...","[[-4.884981308350689e-15, -2.6645352591003757e...",[8.458918697921833e-15]
9,5,0.000001,"[[1.110198600255951e-16], [1.110198600255951e-...","[[-8.881784197001252e-16, -2.220446049250313e-...",[1.998396559451441e-15]
